1. Importação de Bibliotecas e Dados

In [2]:
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands
from pathlib import Path

DATA_PATH = Path("../src/data")
ticker = "PETR4.SA"
df = pd.read_parquet(DATA_PATH / f"{ticker}_historical.parquet")
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

2. Criação de Indicadores Técnicos
- Médias móveis (SMA):

In [3]:
df['SMA_5'] = df['Close'].rolling(window=5).mean()
df['SMA_10'] = df['Close'].rolling(window=10).mean()
df['SMA_20'] = df['Close'].rolling(window=20).mean()

- RSI (Índice de Força Relativa):

In [4]:
df['RSI_14'] = RSIIndicator(df['Close'], window=14).rsi()

- MACD:

In [5]:
macd = MACD(df['Close'])
df['MACD'] = macd.macd()
df['MACD_signal'] = macd.macd_signal()

- Bandas de Bollinger:

In [6]:
bb = BollingerBands(df['Close'], window=20)
df['BB_high'] = bb.bollinger_hband()
df['BB_low'] = bb.bollinger_lband()

3. Criação de Features Temporais
- Dia da semana:

In [7]:
df['weekday'] = df.index.weekday

- Mês do ano:

In [8]:
df['month'] = df.index.month

4. Introdução de Lags
- Features defasadas:

In [9]:
df['Close_lag1'] = df['Close'].shift(1)
df['Close_lag3'] = df['Close'].shift(3)
df['Close_lag5'] = df['Close'].shift(5)

- Volume defasado:

In [10]:
df['Volume_lag1'] = df['Volume'].shift(1)

5. Retornos e Volatilidade
- Retorno percentual diário e acumulado:

In [11]:
df['Return_1d'] = df['Close'].pct_change()
df['Return_5d'] = df['Close'].pct_change(5)
df['Cumulative_Return'] = (1 + df['Return_1d']).cumprod() - 1

- Volatilidade realizada (móvel):

In [12]:
df['Volatility_20d'] = df['Return_1d'].rolling(window=20).std()

- Range verdadeiro (True Range):

In [13]:
df['True_Range'] = np.maximum(
    df['High'] - df['Low'],
    np.maximum(
        abs(df['High'] - df['Close'].shift(1)),
        abs(df['Low'] - df['Close'].shift(1))
    ))

6. Tratamento de Missing Values e Escalonamento
- Remover linhas iniciais com valores faltantes (de indicadores/lags):

In [14]:
df = df.dropna()

- Escalonamento dos dados (opcional, para modelos que exigem):

In [16]:
from sklearn.preprocessing import StandardScaler
features = [col for col in df.columns if col not in ['Close']]
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

7. Salvamento do Dataset Processado
- Salvar dataset para modelagem:

In [17]:
output_file = DATA_PATH / f"{ticker}_features.parquet"
df.to_parquet(output_file, index=True)
print(f"Dataset salvo em: {output_file}")

Dataset salvo em: ..\src\data\PETR4.SA_features.parquet
